## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-23-16-54-57 +0000,nyt,Stephen Miller Cites Children of Immigrants as...,https://www.nytimes.com/2025/12/23/us/politics...
1,2025-12-23-16-47-22 +0000,nyt,"In Epstein Files, Administration Officials Poi...",https://www.nytimes.com/2025/12/23/us/politics...
2,2025-12-23-16-46-19 +0000,nyt,"In Ontario, the Uplifting Case of a Ballet Com...",https://www.nytimes.com/2025/12/23/arts/dance/...
3,2025-12-23-16-44-53 +0000,nyt,"The Confederacy Goes on Trial, Along With Scho...",https://www.nytimes.com/2025/12/23/us/politics...
4,2025-12-23-16-42-47 +0000,cbc,Interim budget officer says he regrets comment...,https://www.cbc.ca/news/politics/interim-pbo-r...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2358/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
12,trump,44
58,new,18
6,epstein,17
190,up,11
680,california,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
148,2025-12-23-05-53-43 +0000,nyt,Trump Announces New ‘Trump Class’ of Warships,https://www.nytimes.com/2025/12/22/us/politics...,115
197,2025-12-23-00-25-00 +0000,wsj,President Trump said he wants to pressure Amer...,https://www.wsj.com/politics/policy/trump-targ...,106
268,2025-12-22-19-16-47 +0000,nypost,Bill Clinton accuses Trump’s DOJ of cover-up i...,https://nypost.com/2025/12/22/us-news/bill-cli...,104
46,2025-12-23-14-57-01 +0000,nypost,Trump flew on Epstein’s private jet ‘at least ...,https://nypost.com/2025/12/23/us-news/trump-fl...,101
1,2025-12-23-16-47-22 +0000,nyt,"In Epstein Files, Administration Officials Poi...",https://www.nytimes.com/2025/12/23/us/politics...,84


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
148,115,2025-12-23-05-53-43 +0000,nyt,Trump Announces New ‘Trump Class’ of Warships,https://www.nytimes.com/2025/12/22/us/politics...
268,60,2025-12-22-19-16-47 +0000,nypost,Bill Clinton accuses Trump’s DOJ of cover-up i...,https://nypost.com/2025/12/22/us-news/bill-cli...
252,44,2025-12-22-20-39-00 +0000,wsj,Former Vice President Mike Pence’s political g...,https://www.wsj.com/politics/elections/top-her...
157,36,2025-12-23-03-56-42 +0000,bbc,Brown University places its police chief on le...,https://www.bbc.com/news/articles/cy0p7d74ywgo...
237,35,2025-12-22-21-55-00 +0000,wsj,A coalition of Democratic state attorneys gene...,https://www.wsj.com/politics/policy/democratic...
117,35,2025-12-23-10-04-47 +0000,bbc,Call of Duty co-creator Vince Zampella dies in...,https://www.bbc.com/news/articles/cx25rled0ylo...
204,33,2025-12-22-23-38-39 +0000,nypost,Warner Bros. Discovery to review Larry Ellison...,https://nypost.com/2025/12/22/media/warner-bro...
254,32,2025-12-22-20-30-03 +0000,latimes,Instacart ends AI pricing test that charged sh...,https://www.latimes.com/business/story/2025-12...
158,31,2025-12-23-03-41-55 +0000,cbc,60 Minutes prison segment postponed by CBS bri...,https://www.cbc.ca/news/canada/cbs-60-minutes-...
41,30,2025-12-23-15-09-18 +0000,nypost,US economy unexpectedly surges 4.3% in third q...,https://nypost.com/2025/12/23/business/us-econ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
